In [2]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [5]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_4/KHCDPRGPTTL1/KHCDPRGPTTL1_final.vcf', sep="\t", comment="#", header= None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,chr1,14653,rs62635297,C,T,.,PASS,ADP=14;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:30:14:14:6:8:57.14%:9.6618E-4:55:44:6:0:8:0
1,chr1,14937,rs11807796,T,C,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:23:24:24:17:7:29.17%:4.7006E-3:53:45:14:3:2:5
2,chr1,16495,rs3210724,G,C,.,PASS,ADP=13;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:42:13:13:3:10:76.92%:5.3843E-5:37:40:3:0:10:0
3,chr1,24786,rs11266852,G,A,.,PASS,ADP=49;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:93:49:49:24:25:51.02%:4.6489E-10:54:58:19:...
4,chr1,24912,rs1382803901,G,A,.,PASS,ADP=25;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:53:25:25:11:14:56%:4.7528E-6:56:46:6:5:6:8
...,...,...,...,...,...,...,...,...,...,...
56407,chrY,57209184,.,T,C,.,PASS,ADP=26;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:26:26:26:18:8:30.77%:2.076E-3:51:49:14:4:7:1
56408,chrY,57209365,.,T,C,.,PASS,ADP=19;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:20:19:19:13:6:31.58%:9.828E-3:62:42:11:2:1:5
56409,chrY,57209591,.,A,G,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=G|non_coding_t...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:31:8:8:1:7:87.5%:6.993E-4:74:42:1:0:7:0
56410,chrY,57209613,.,A,G,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=G|non_coding_t...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:41:8:8:0:8:100%:7.77E-5:0:50:0:0:6:2


In [8]:
vcf[vcf['rsID'] == 'rs1921']

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
60,chr1,1014228,rs1921,G,A,.,PASS,"ADP=38;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6611,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:56:38:38:22:16:42.11%:2.0535E-6:53:48:19:3...


In [22]:
vcf = pd.read_csv(r'C:/Users/Dell/Downloads/KHHSPTGPTTL34_final_Depth.vcf', comment="#", sep="\t", header=0, low_memory=False)
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,RDF,RDR,ADF,ADR,FREQ,PVAL
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,35,35,28,7,20%,5.609500e-03,28,0,7,0
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,76,29,29,10,19,65.52%,2.114400e-08,5,5,16,3
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",1/1,87,26,26,6,20,76.92%,1.827300e-09,4,2,15,5
3,chr1,16495,rs3210724,G,C,.,PASS,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,71,17,17,2,15,88.24%,7.327700e-08,2,0,15,0
4,chr1,17365,rs369606208,C,G,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,0/1,52,62,62,46,16,25.81%,5.029900e-06,28,18,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47664,chrY,57208581,.,G,C,.,PASS,ADP=20;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,46,20,20,8,12,60%,2.254800e-05,8,0,12,0
47665,chrY,57209184,.,T,C,.,PASS,ADP=33;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,33,33,33,23,10,30.3%,4.387200e-04,23,0,10,0
47666,chrY,57209591,.,A,G,.,PASS,ADP=14;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,30,14,14,6,8,57.14%,9.661800e-04,4,2,7,1
47667,chrY,57209613,.,A,G,.,PASS,ADP=15;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,29,15,15,7,8,53.33%,1.099500e-03,5,2,7,1


In [23]:
vcf.columns

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT',
       'GQ', 'SDP', 'DP', 'RD', 'AD', 'RDF', 'RDR', 'ADF', 'ADR', 'FREQ',
       'PVAL'],
      dtype='object')

In [24]:
vcf.columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO',
               'Genotype', 'Genotype Quality', 'Raw Read Depth as reported by SAMtools', 
               'Quality Read Depth of bases with Phred score >= 15', 
               'Depth of reference-supporting bases (reads1)', 'Depth of variant-supporting bases (reads2)', 
               'Depth of reference-supporting bases on forward strand (reads1plus)',
               'Depth of reference-supporting bases on reverse strand (reads1minus)', 
               'Depth of variant-supporting bases on forward strand (reads2plus)', 
               'Depth of variant-supporting bases on reverse strand (reads2minus)', 
               'Variant allele frequency', 'P-value from Fishers Exact Test']
vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,Raw Read Depth as reported by SAMtools,Quality Read Depth of bases with Phred score >= 15,Depth of reference-supporting bases (reads1),Depth of variant-supporting bases (reads2),Depth of reference-supporting bases on forward strand (reads1plus),Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,35,35,28,7,20%,5.609500e-03,28,0,7,0
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,76,29,29,10,19,65.52%,2.114400e-08,5,5,16,3
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",1/1,87,26,26,6,20,76.92%,1.827300e-09,4,2,15,5
3,chr1,16495,rs3210724,G,C,.,PASS,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,71,17,17,2,15,88.24%,7.327700e-08,2,0,15,0
4,chr1,17365,rs369606208,C,G,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,0/1,52,62,62,46,16,25.81%,5.029900e-06,28,18,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47664,chrY,57208581,.,G,C,.,PASS,ADP=20;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,46,20,20,8,12,60%,2.254800e-05,8,0,12,0
47665,chrY,57209184,.,T,C,.,PASS,ADP=33;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,33,33,33,23,10,30.3%,4.387200e-04,23,0,10,0
47666,chrY,57209591,.,A,G,.,PASS,ADP=14;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,30,14,14,6,8,57.14%,9.661800e-04,4,2,7,1
47667,chrY,57209613,.,A,G,.,PASS,ADP=15;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,29,15,15,7,8,53.33%,1.099500e-03,5,2,7,1


In [25]:
vcf.columns

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO',
       'Genotype', 'Genotype Quality',
       'Raw Read Depth as reported by SAMtools',
       'Quality Read Depth of bases with Phred score >= 15',
       'Depth of reference-supporting bases (reads1)',
       'Depth of variant-supporting bases (reads2)',
       'Depth of reference-supporting bases on forward strand (reads1plus)',
       'Depth of reference-supporting bases on reverse strand (reads1minus)',
       'Depth of variant-supporting bases on forward strand (reads2plus)',
       'Depth of variant-supporting bases on reverse strand (reads2minus)',
       'Variant allele frequency', 'P-value from Fishers Exact Test'],
      dtype='object')

In [26]:
vcf["New_Column"] = vcf["INFO"].str.extract(r'^(.*?)(?=;?CSQ)')
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Quality Read Depth of bases with Phred score >= 15,Depth of reference-supporting bases (reads1),Depth of variant-supporting bases (reads2),Depth of reference-supporting bases on forward strand (reads1plus),Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,New_Column
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,35,28,7,20%,5.609500e-03,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,76,...,29,10,19,65.52%,2.114400e-08,5,5,16,3,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",1/1,87,...,26,6,20,76.92%,1.827300e-09,4,2,15,5,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0...."
3,chr1,16495,rs3210724,G,C,.,PASS,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,71,...,17,2,15,88.24%,7.327700e-08,2,0,15,0,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...
4,chr1,17365,rs369606208,C,G,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,0/1,52,...,62,46,16,25.81%,5.029900e-06,28,18,15,1,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47664,chrY,57208581,.,G,C,.,PASS,ADP=20;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,46,...,20,8,12,60%,2.254800e-05,8,0,12,0,ADP=20;WT=0;HET=1;HOM=0;NC=0
47665,chrY,57209184,.,T,C,.,PASS,ADP=33;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,33,...,33,23,10,30.3%,4.387200e-04,23,0,10,0,ADP=33;WT=0;HET=1;HOM=0;NC=0
47666,chrY,57209591,.,A,G,.,PASS,ADP=14;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,30,...,14,6,8,57.14%,9.661800e-04,4,2,7,1,ADP=14;WT=0;HET=1;HOM=0;NC=0
47667,chrY,57209613,.,A,G,.,PASS,ADP=15;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,29,...,15,7,8,53.33%,1.099500e-03,5,2,7,1,ADP=15;WT=0;HET=1;HOM=0;NC=0


In [27]:
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Depth of reference-supporting bases on forward strand (reads1plus),Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,New_Column,HET,HOM,Zygosity
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,20%,5.609500e-03,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,76,...,65.52%,2.114400e-08,5,5,16,3,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",1/1,87,...,76.92%,1.827300e-09,4,2,15,5,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",0,1,Homozygous
3,chr1,16495,rs3210724,G,C,.,PASS,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,71,...,88.24%,7.327700e-08,2,0,15,0,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,0,1,Homozygous
4,chr1,17365,rs369606208,C,G,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,0/1,52,...,25.81%,5.029900e-06,28,18,15,1,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,1,0,Heterozygous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47664,chrY,57208581,.,G,C,.,PASS,ADP=20;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,46,...,60%,2.254800e-05,8,0,12,0,ADP=20;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous
47665,chrY,57209184,.,T,C,.,PASS,ADP=33;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,33,...,30.3%,4.387200e-04,23,0,10,0,ADP=33;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous
47666,chrY,57209591,.,A,G,.,PASS,ADP=14;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,30,...,57.14%,9.661800e-04,4,2,7,1,ADP=14;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous
47667,chrY,57209613,.,A,G,.,PASS,ADP=15;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,29,...,53.33%,1.099500e-03,5,2,7,1,ADP=15;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous


In [28]:
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,New_Column,HET,HOM,Zygosity,CSQ
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,5.609500e-03,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,76,...,2.114400e-08,5,5,16,3,ADP=29;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,G|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",1/1,87,...,1.827300e-09,4,2,15,5,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5178,0....",0,1,Homozygous,G|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
3,chr1,16495,rs3210724,G,C,.,PASS,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,71,...,7.327700e-08,2,0,15,0,ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,0,1,Homozygous,C|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
4,chr1,17365,rs369606208,C,G,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,0/1,52,...,5.029900e-06,28,18,15,1,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,1,0,Heterozygous,G|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47664,chrY,57208581,.,G,C,.,PASS,ADP=20;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,46,...,2.254800e-05,8,0,12,0,ADP=20;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous,C|non_coding_transcript_exon_variant|MODIFIER|...
47665,chrY,57209184,.,T,C,.,PASS,ADP=33;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,33,...,4.387200e-04,23,0,10,0,ADP=33;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous,C|non_coding_transcript_exon_variant|MODIFIER|...
47666,chrY,57209591,.,A,G,.,PASS,ADP=14;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,30,...,9.661800e-04,4,2,7,1,ADP=14;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous,G|non_coding_transcript_exon_variant|MODIFIER|...
47667,chrY,57209613,.,A,G,.,PASS,ADP=15;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,29,...,1.099500e-03,5,2,7,1,ADP=15;WT=0;HET=1;HOM=0;NC=0,1,0,Heterozygous,G|non_coding_transcript_exon_variant|MODIFIER|...


In [29]:
vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,New_Column,HET,HOM,Zygosity,CSQ,csq
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,ADP=8;WT=0;HET=0;HOM=1;NC=0,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,ADP=8;WT=0;HET=0;HOM=1;NC=0,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,ADP=8;WT=0;HET=0;HOM=1;NC=0,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,ADP=8;WT=0;HET=0;HOM=1;NC=0,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...


In [30]:
vcf['Allele'] = vcf['csq'].str.split('|').str[0]
vcf['Consequence'] = vcf['csq'].str.split('|').str[1]
vcf['IMPACT'] = vcf['csq'].str.split('|').str[2]
vcf['SYMBOL'] = vcf['csq'].str.split('|').str[3]
vcf['Gene'] = vcf['csq'].str.split('|').str[4]
vcf['Feature_type'] = vcf['csq'].str.split('|').str[5]
vcf['Feature'] = vcf['csq'].str.split('|').str[6]
vcf['BIOTYPE'] = vcf['csq'].str.split('|').str[7]
vcf['EXON'] = vcf['csq'].str.split('|').str[8]
vcf['INTRON'] = vcf['csq'].str.split('|').str[9]
vcf['HGVSc'] = vcf['csq'].str.split('|').str[10]
vcf['HGVSp'] = vcf['csq'].str.split('|').str[11]
vcf['cDNA_position'] = vcf['csq'].str.split('|').str[12]
vcf['CDS_position'] = vcf['csq'].str.split('|').str[13]
vcf['Protein_position'] = vcf['csq'].str.split('|').str[14]
vcf['Amino_acids'] = vcf['csq'].str.split('|').str[15]
vcf['Codons'] = vcf['csq'].str.split('|').str[16]
vcf['Existing_variation'] = vcf['csq'].str.split('|').str[17]
vcf['DISTANCE'] = vcf['csq'].str.split('|').str[18]
vcf['STRAND'] = vcf['csq'].str.split('|').str[19]
vcf['FLAGS'] = vcf['csq'].str.split('|').str[20]
vcf['VARIANT_CLASS'] = vcf['csq'].str.split('|').str[21]
vcf['SYMBOL_SOURCE'] = vcf['csq'].str.split('|').str[22]
vcf['HGNC_ID'] = vcf['csq'].str.split('|').str[23]
vcf['CANONICAL'] = vcf['csq'].str.split('|').str[24]
vcf['MANE_SELECT'] = vcf['csq'].str.split('|').str[25]
vcf['MANE_PLUS_CLINICAL'] = vcf['csq'].str.split('|').str[26]
vcf['TSL'] = vcf['csq'].str.split('|').str[27]
vcf['APPRIS'] = vcf['csq'].str.split('|').str[28]
vcf['CCDS'] = vcf['csq'].str.split('|').str[29]
vcf['ENSP'] = vcf['csq'].str.split('|').str[30]
vcf['SWISSPROT'] = vcf['csq'].str.split('|').str[31]
vcf['TREMBL'] = vcf['csq'].str.split('|').str[32]
vcf['UNIPARC'] = vcf['csq'].str.split('|').str[33]
vcf['UNIPROT_ISOFORM'] = vcf['csq'].str.split('|').str[34]
vcf['SOURCE'] = vcf['csq'].str.split('|').str[35]
vcf['GENE_PHENO'] = vcf['csq'].str.split('|').str[36]
vcf['SIFT'] = vcf['csq'].str.split('|').str[37]
vcf['PolyPhen'] = vcf['csq'].str.split('|').str[38]
vcf['DOMAINS'] = vcf['csq'].str.split('|').str[39]
vcf['miRNA'] = vcf['csq'].str.split('|').str[40]
vcf['HGVS_OFFSET'] = vcf['csq'].str.split('|').str[41]
vcf['AF'] = vcf['csq'].str.split('|').str[42]
vcf['AFR_AF'] = vcf['csq'].str.split('|').str[43]
vcf['AMR_AF'] = vcf['csq'].str.split('|').str[44]
vcf['EAS_AF'] = vcf['csq'].str.split('|').str[45]
vcf['EUR_AF'] = vcf['csq'].str.split('|').str[46]
vcf['SAS_AF'] = vcf['csq'].str.split('|').str[47]
vcf['gnomADe_AF'] = vcf['csq'].str.split('|').str[48]
vcf['gnomADe_AFR_AF'] = vcf['csq'].str.split('|').str[49]
vcf['gnomADe_AMR_AF'] = vcf['csq'].str.split('|').str[50]
vcf['gnomADe_ASJ_AF'] = vcf['csq'].str.split('|').str[51]
vcf['gnomADe_EAS_AF'] = vcf['csq'].str.split('|').str[52]
vcf['gnomADe_FIN_AF'] = vcf['csq'].str.split('|').str[53]
vcf['gnomADe_NFE_AF'] = vcf['csq'].str.split('|').str[54]
vcf['gnomADe_OTH_AF'] = vcf['csq'].str.split('|').str[55]
vcf['gnomADe_SAS_AF'] = vcf['csq'].str.split('|').str[56]
vcf['gnomADg_AF'] = vcf['csq'].str.split('|').str[57]
vcf['gnomADg_AFR_AF'] = vcf['csq'].str.split('|').str[58]
vcf['gnomADg_AMI_AF'] = vcf['csq'].str.split('|').str[59]
vcf['gnomADg_AMR_AF'] = vcf['csq'].str.split('|').str[60]
vcf['gnomADg_ASJ_AF'] = vcf['csq'].str.split('|').str[61]
vcf['gnomADg_EAS_AF'] = vcf['csq'].str.split('|').str[62]
vcf['gnomADg_FIN_AF'] = vcf['csq'].str.split('|').str[63]
vcf['gnomADg_MID_AF'] = vcf['csq'].str.split('|').str[64]
vcf['gnomADg_NFE_AF'] = vcf['csq'].str.split('|').str[65]
vcf['gnomADg_OTH_AF'] = vcf['csq'].str.split('|').str[66]
vcf['gnomADg_SAS_AF'] = vcf['csq'].str.split('|').str[67]
vcf['MAX_AF'] = vcf['csq'].str.split('|').str[68]
vcf['MAX_AF_POPS'] = vcf['csq'].str.split('|').str[69]
vcf['CLIN_SIG'] = vcf['csq'].str.split('|').str[70]
vcf['SOMATIC'] = vcf['csq'].str.split('|').str[71]
vcf['PHENO'] = vcf['csq'].str.split('|').str[72]
vcf['PUBMED'] = vcf['csq'].str.split('|').str[73]
vcf['MOTIF_NAME'] = vcf['csq'].str.split('|').str[74]
vcf['MOTIF_POS'] = vcf['csq'].str.split('|').str[75]
vcf['HIGH_INF_POS'] = vcf['csq'].str.split('|').str[76]
vcf['MOTIF_SCORE_CHANGE'] = vcf['csq'].str.split('|').str[77]
vcf['TRANSCRIPTION_FACTORS'] = vcf['csq'].str.split('|').str[78]
vcf['ClinVar'] = vcf['csq'].str.split('|').str[79]
vcf['ClinVar_CLNSIG'] = vcf['csq'].str.split('|').str[80]
vcf['ClinVar_CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['ClinVar_CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,


In [31]:
vcf_1 = vcf.copy()
vcf_1

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,


In [33]:
# Create empty columns
columns = ['ADP', 'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD', 'dbSNPBuildID', 'SLO',
           'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON', 'RS', 'RV', 'TPA', 'CFL', 'GNO',
           'VLD', 'ASP', 'ASS', 'REF', 'U3', 'U5', 'TOPMED', 'WGT', 'MTP', 'LSD', 'NOC',
           'DSS', 'SYN', 'KGPhase3', 'CAF', 'VC', 'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO',
           'GENEINFO', 'INT', 'G5', 'OM', 'PMC', 'SSR', 'RSPOS', 'HD', 'PM', 'ClinVar',
           'ClinVar_CLNSIG']

for col in columns:
    vcf_1[col] = ''

# Populate columns based on 'info' values
for i, row in vcf_1.iterrows():
    info = row['New_Column']
    items = info.split(';')
    for item in items:
        key_value = item.split('=')
        key = key_value[0]
        if key in columns:
            if len(key_value) > 1:
                value = key_value[1]
                vcf_1.at[i, key] = f"{key}={value}"
            else:
                vcf_1.at[i, key] = key
        else:
            vcf_1.at[i, key] = 'null'

# Print the resulting DataFrame
vcf_1

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,


In [52]:
vcf_1.ID.value_counts()

ID
.              6677
rs28529458      149
rs12624152      145
rs958           122
rs183956        112
               ... 
rs4768269         1
rs749770969       1
rs4768275         1
rs2933356         1
rs6568353         1
Name: count, Length: 44733, dtype: int64

In [67]:
vcf_1['Protein Position and Amino Acid'] = vcf_1['Amino_acids'].str[0] + vcf_1['Protein_position'] + np.where(vcf_1['Amino_acids'].str[-1] == vcf_1['Amino_acids'].str[0], '', vcf_1['Amino_acids'].str[-1])
vcf_1

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,Raw Read Depth as reported by SAMtools,Quality Read Depth of bases with Phred score >= 15,Depth of reference-supporting bases (reads1),Depth of variant-supporting bases (reads2),Depth of reference-supporting bases on forward strand (reads1plus),Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,New_Column,HET,HOM,Zygosity,CSQ,csq,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN,ADP,WT,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,Protein Position and Amino Acid
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,35,35,28,7,20%,0.005609,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,HET=1,HOM=0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305,transcribed_unprocessed_pseudogene,,,,,,,,,,rs17855866,1082,1,,SNV,HGNC,HGNC:37102,YES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.00162,0.005487,0,0.001186,0.0002987,0.0001935,0,0,6.055e-05,0.00203,0.0002195,0.005487,gnomADg_AFR,,,,,,,,,,,,,,ADP=35,WT=0,NC=0,,,,,dbSNPBuildID=123,,,R3,,,,,,RS=17855866,,,,,,ASP,,,,"TOPMED=0.99442533129459734,0.00553484964322120...",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050000040005000002000100,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,,NaN
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,35,35,28,7,20%,0.005609,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,HET=1,HOM=0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,A,downstream_gene_variant,MODIFIER,DDX11L2,ENSG00000290825,Transcript,ENST00000456328,lncRNA,,,,,,,,,,rs17855866,343,1,,SNV,EntrezGene,,YES,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.00162,0.005487,0,0.001186,0.0002987,0.0001935,0,0,6.055e-05,0.00203,0.0002195,0.005487,gnomADg_AFR,,,,,,,,,,,,,,ADP=35,WT=0,NC=0,,,,,dbSNPBuildID=123,,,R3,,,,,,RS=17855866,,,,,,ASP,,,,"TOPMED=0.99442533129459734,0.00553484964322120...",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050000040005000002000100,SAO=0,GENEINFO=DDX11L1:100287102|WASH7P:653635,,,,,SSR=0,RSPOS=14752,,,NaN
0,chr1,14752,rs17855866,,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,35,35,28,7,20%,0.005609,28,0,7,0,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,HET=1,HOM=0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,A,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,ENSG00000227232,Transcript,ENST00000488147,unprocessed_pseudogene,,10/10,ENST00000488147.1:n.1254-251C>T,,,,,,,rs17855866,,-1,,SNV,HGNC,HGNC:38034,YES,,,,,,,,,,,,,,,,,,,,,,,,,,

In [71]:
vcf_1[vcf_1['Protein Position and Amino Acid'] == 'I391']

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,Raw Read Depth as reported by SAMtools,Quality Read Depth of bases with Phred score >= 15,Depth of reference-supporting bases (reads1),Depth of variant-supporting bases (reads2),Depth of reference-supporting bases on forward strand (reads1plus),Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,New_Column,HET,HOM,Zygosity,CSQ,csq,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN,ADP,WT,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,Protein Position and Amino Acid
18541,chr17,21007891,rs11541311,REF,T,.,PASS,"ADP=11;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.8442,.,...",1/1,58,11,11,0,11,100%,0.000001,0,0,7,4,"ADP=11;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.8442,.,...",HET=0,HOM=1,Homozygous,T|synonymous_variant|LOW|USP22|ENSG00000124422...,T|synonymous_variant|LOW|USP22|ENSG00000124422...,T,synonymous_variant,LOW,USP22,ENSG00000124422,Transcript,ENST00000537526,protein_coding,9/13,,ENST00000537526.6:c.1173C>A,ENSP00000440950.2:p.Ile391%3D,1173,1173,391,I,atC/atA,rs11541311&COSV54945859,,-1,,SNV,HGNC,HGNC:12621,,,,1,,,ENSP00000440950,Q9UPT9.179,,UPI00016397EF,Q9UPT9-2,,,,,Gene3D:3.90.70.10&Pfam:PF00443&PROSITE_profile...,,,0.1558,0.0159,0.1326,0.1667,0.2465,0.2566,0.1974,0.04178,0.1105,0.176,0.1724,0.3131,0.2181,0.1977,0.2381,0.1705,0.04537,0.2303,0.1462,0.1723,0.1723,0.3116,0.1487,0.224,0.1644,0.2474,0.3131,gnomADe_FIN,,0&1,0&1,,,,,,,,,,,ADP=11,WT=0,NC=0,,,S3D,,dbSNPBuildID=120,SLO,,,,,NSM,,COMMON=1,RS=11541311,RV,,,GNO,VLD,ASP,,,,"TOPMED=0.84234040519877675,0.00013538481141692...",WGT=1,,,,,SYN,KGPhase3,"CAF=0.8442,.,.,0.1558",VC=SNV,,KGPhase1,,VP=0x050300000b05150536000100,SAO=0,GENEINFO=USP22:23326,,G5,,,SSR=0,RSPOS=21007891,HD,,I391


In [68]:
vcf_1['Protein Position and Amino Acid'].value_counts()


Protein Position and Amino Acid
A28T     58
L26V     58
A15      57
M13V     52
S38      50
         ..
A84V      1
A17P      1
Y107      1
I391      1
S268G     1
Name: count, Length: 27378, dtype: int64

In [55]:
vcf_1[vcf_1['ID'] == 'rs183956'].INFO.iloc[0]

'ADP=19;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.01278,0.9872;COMMON=1;GENEINFO=CRELD1:78987|IL17RC:84818;GNO;KGPhase1;KGPhase3;R5;REF;RS=183956;RSPOS=9933563;RV;SAO=0;SLO;SSR=0;SYN;TOPMED=0.01415965851172273,0.98584034148827726;VC=SNV;VLD;VP=0x050100020305140136000100;WGT=1;dbSNPBuildID=79;CSQ=G|synonymous_variant|LOW|IL17RC|ENSG00000163702|Transcript|ENST00000295981|protein_coding|19/19||ENST00000295981.7:c.2346C>G|ENSP00000295981.3:p.Gly782%3D|2564|2346|782|G|ggC/ggG|rs183956||1||SNV|HGNC|HGNC:18358||||2|A2|CCDS2590.1|ENSP00000295981|Q8NAC3.156||UPI000013E2E3|Q8NAC3-1||1|||Low_complexity_(Seg):seg&MobiDB_lite:mobidb-lite&AFDB-ENSP_mappings:AF-Q8NAC3-F1.A|||0.9872|0.9728|0.9942|0.999|0.9831|0.9939|0.9894|0.9779|0.9915|0.9899|0.9969|0.9979|0.9877|0.9916|0.9874|0.9859|0.977|1|0.9884|0.9885|0.9975|0.998|0.9842|0.9875|0.9842|0.9897|1|gnomADg_AMI|benign||1|||||||1164864|Benign|criteria_provided&_multiple_submitters&_no_conflicts|Candidiasis&_familial&_9&not_provided,G|upstream_gene_variant|MODIFIE

In [36]:
vcf_1.to_csv(r'C:/Users/Dell/Downloads/VCF_all_columns_split.csv', sep='\t', index=False)

In [35]:
print(vcf_1.columns)

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO',
       'Genotype', 'Genotype Quality',
       ...
       'SAO', 'GENEINFO', 'INT', 'G5', 'OM', 'PMC', 'SSR', 'RSPOS', 'HD',
       'PM'],
      dtype='object', length=159)


In [13]:
x = vcf[vcf['ID'] == 'rs6672356']

In [14]:
x.to_excel(r'C:/Users/Dell/Downloads/rs6672356.xlsx', index=False)

In [9]:
vcf['Consequence'] = vcf['csq'].str.extract(r'\|([^|]+)\|')
vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,HET,HOM,Zygosity,CSQ,csq,Consequence
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,28,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant


In [10]:
vcf.Consequence.isnull().sum()

0

In [11]:
vcf['Impact'] = vcf['csq'].str.split('|').str[2]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test,HET,HOM,Zygosity,CSQ,csq,Consequence,Impact
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant,MODIFIER
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant,MODIFIER
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant,MODIFIER
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant,MODIFIER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant,MODIFIER
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER


In [12]:
vcf['Gene_Name'] = vcf['csq'].str.split('|').str[3]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Variant allele frequency,P-value from Fishers Exact Test,HET,HOM,Zygosity,CSQ,csq,Consequence,Impact,Gene_Name
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant,MODIFIER,DDX11L1
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant,MODIFIER,DDX11L2
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant,MODIFIER,MIR6859-1
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,7,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant,MODIFIER,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant,MODIFIER,WASH6P
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,6,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH6P


In [13]:
vcf['Biotype'] = vcf['csq'].str.split('|').str[7]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,P-value from Fishers Exact Test,HET,HOM,Zygosity,CSQ,csq,Consequence,Impact,Gene_Name,Biotype
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant,MODIFIER,DDX11L1,transcribed_unprocessed_pseudogene
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant,MODIFIER,DDX11L2,lncRNA
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,unprocessed_pseudogene
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant,MODIFIER,MIR6859-1,miRNA
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant,MODIFIER,,enhancer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant,MODIFIER,WASH6P,retained_intron
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH6P,retained_intron


In [14]:
vcf['Strand'] = vcf['csq'].str.split('|').str[19]
vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,HET,HOM,Zygosity,CSQ,csq,Consequence,Impact,Gene_Name,Biotype,Strand
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant,MODIFIER,DDX11L1,transcribed_unprocessed_pseudogene,1
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant,MODIFIER,DDX11L2,lncRNA,1
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,unprocessed_pseudogene,-1
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant,MODIFIER,MIR6859-1,miRNA,-1
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant,MODIFIER,,enhancer,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant,MODIFIER,WASH6P,retained_intron,1
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,0,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH6P,retained_intron,1


In [15]:
vcf['Clinical Significance'] = vcf['csq'].str.split('|').str[80]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,HOM,Zygosity,CSQ,csq,Consequence,Impact,Gene_Name,Biotype,Strand,Clinical Significance
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant,MODIFIER,DDX11L1,transcribed_unprocessed_pseudogene,1,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant,MODIFIER,DDX11L2,lncRNA,1,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,unprocessed_pseudogene,-1,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant,MODIFIER,MIR6859-1,miRNA,-1,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant,MODIFIER,,enhancer,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant,MODIFIER,WASH6P,retained_intron,1,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,Homozygous,T|intron_variant&non_coding_transcript_variant...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH6P,retained_intron,1,


In [16]:
vcf['Clinical Significance'].value_counts()

Clinical Significance
                                                            261106
Benign                                                      101428
Benign/Likely_benign                                          4386
Likely_benign                                                 1806
Uncertain_significance                                        1110
Conflicting_interpretations_of_pathogenicity                   822
risk_factor                                                    216
association                                                    216
drug_response                                                  180
not_provided                                                    60
Uncertain_significance&drug_response                            28
protective                                                      19
Conflicting_interpretations_of_pathogenicity&other              19
Benign&drug_response                                            18
Benign&association                      

In [17]:
vcf['Codons_position'] = vcf['csq'].str.split('|').str[10]
vcf['Protein'] = vcf['csq'].str.split('|').str[11]
vcf['Codon'] = vcf['csq'].str.split('|').str[16]
vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,csq,Consequence,Impact,Gene_Name,Biotype,Strand,Clinical Significance,Codons_position,Protein,Codon
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,downstream_gene_variant,MODIFIER,DDX11L1,transcribed_unprocessed_pseudogene,1,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,downstream_gene_variant,MODIFIER,DDX11L2,lncRNA,1,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,A|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,unprocessed_pseudogene,-1,,ENST00000488147.1:n.1254-251C>T,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,downstream_gene_variant,MODIFIER,MIR6859-1,miRNA,-1,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,A|regulatory_region_variant|MODIFIER|||Regulat...,regulatory_region_variant,MODIFIER,,enhancer,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1,,ENST00000484415.6:n.813C>T,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,T|non_coding_transcript_exon_variant|MODIFIER|...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1,,ENST00000492963.6:n.1157C>T,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,T|downstream_gene_variant|MODIFIER|WASH6P|ENSG...,downstream_gene_variant,MODIFIER,WASH6P,retained_intron,1,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,T|intron_variant&non_coding_transcript_variant...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH6P,retained_intron,1,,ENST00000496301.6:n.94+36C>T,,


In [18]:
vcf['Clinvar ID'] = vcf['csq'].str.split('|').str[79]
vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Consequence,Impact,Gene_Name,Biotype,Strand,Clinical Significance,Codons_position,Protein,Codon,Clinvar ID
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,downstream_gene_variant,MODIFIER,DDX11L1,transcribed_unprocessed_pseudogene,1,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,downstream_gene_variant,MODIFIER,DDX11L2,lncRNA,1,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,unprocessed_pseudogene,-1,,ENST00000488147.1:n.1254-251C>T,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,downstream_gene_variant,MODIFIER,MIR6859-1,miRNA,-1,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,regulatory_region_variant,MODIFIER,,enhancer,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1,,ENST00000484415.6:n.813C>T,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,non_coding_transcript_exon_variant,MODIFIER,WASH6P,retained_intron,1,,ENST00000492963.6:n.1157C>T,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,downstream_gene_variant,MODIFIER,WASH6P,retained_intron,1,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,intron_variant&non_coding_transcript_variant,MODIFIER,WASH6P,retained_intron,1,,ENST00000496301.6:n.94+36C>T,,,


In [19]:
vcf['Exon'] = vcf['csq'].str.split('|').str[8]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Impact,Gene_Name,Biotype,Strand,Clinical Significance,Codons_position,Protein,Codon,Clinvar ID,Exon
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,MODIFIER,DDX11L1,transcribed_unprocessed_pseudogene,1,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,MODIFIER,DDX11L2,lncRNA,1,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,MODIFIER,WASH7P,unprocessed_pseudogene,-1,,ENST00000488147.1:n.1254-251C>T,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,MODIFIER,MIR6859-1,miRNA,-1,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,MODIFIER,,enhancer,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,MODIFIER,WASH6P,retained_intron,1,,ENST00000484415.6:n.813C>T,,,,5/5
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,MODIFIER,WASH6P,retained_intron,1,,ENST00000492963.6:n.1157C>T,,,,5/6
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,MODIFIER,WASH6P,retained_intron,1,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,MODIFIER,WASH6P,retained_intron,1,,ENST00000496301.6:n.94+36C>T,,,,


In [20]:
vcf['Existing variant'] = vcf['csq'].str.split('|').str[17]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Gene_Name,Biotype,Strand,Clinical Significance,Codons_position,Protein,Codon,Clinvar ID,Exon,Existing variant
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,DDX11L1,transcribed_unprocessed_pseudogene,1,,,,,,,rs17855866
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,DDX11L2,lncRNA,1,,,,,,,rs17855866
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,WASH7P,unprocessed_pseudogene,-1,,ENST00000488147.1:n.1254-251C>T,,,,,rs17855866
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,MIR6859-1,miRNA,-1,,,,,,,rs17855866
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,enhancer,,,,,,,,rs17855866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,WASH6P,retained_intron,1,,ENST00000484415.6:n.813C>T,,,,5/5,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,WASH6P,retained_intron,1,,ENST00000492963.6:n.1157C>T,,,,5/6,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,WASH6P,retained_intron,1,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,WASH6P,retained_intron,1,,ENST00000496301.6:n.94+36C>T,,,,,


In [21]:
vcf['CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Strand,Clinical Significance,Codons_position,Protein,Codon,Clinvar ID,Exon,Existing variant,CLNREVSTAT,CLNDN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,,,,,,,rs17855866,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,1,,,,,,,rs17855866,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,-1,,ENST00000488147.1:n.1254-251C>T,,,,,rs17855866,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,-1,,,,,,,rs17855866,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,rs17855866,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,,ENST00000484415.6:n.813C>T,,,,5/5,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,,ENST00000492963.6:n.1157C>T,,,,5/6,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,1,,ENST00000496301.6:n.94+36C>T,,,,,,,


In [22]:
vcf['Position_14'] = vcf['csq'].str.split('|').str[14]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Clinical Significance,Codons_position,Protein,Codon,Clinvar ID,Exon,Existing variant,CLNREVSTAT,CLNDN,Position_14
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,rs17855866,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,rs17855866,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,ENST00000488147.1:n.1254-251C>T,,,,,rs17855866,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,rs17855866,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,rs17855866,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,ENST00000484415.6:n.813C>T,,,,5/5,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,ENST00000492963.6:n.1157C>T,,,,5/6,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,ENST00000496301.6:n.94+36C>T,,,,,,,,


In [23]:
vcf['Position_15'] = vcf['csq'].str.split('|').str[15]

# Format the values as "W170R"
vcf['Protein Position and Amino Acid'] = vcf['Position_15'].str[0] + vcf['Position_14'] + vcf['Position_15'].str[-1]
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Protein,Codon,Clinvar ID,Exon,Existing variant,CLNREVSTAT,CLNDN,Position_14,Position_15,Protein Position and Amino Acid
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,rs17855866,,,,,NaN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,rs17855866,,,,,NaN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,rs17855866,,,,,NaN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,rs17855866,,,,,NaN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,rs17855866,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,5/5,,,,,,NaN
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,5/6,,,,,,NaN
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,NaN
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,NaN


In [25]:
vcf.columns

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO',
       'Genotype', 'Genotype Quality',
       'Raw Read Depth as reported by SAMtools',
       'Quality Read Depth of bases with Phred score >= 15',
       'Depth of reference-supporting bases (reads1)',
       'Depth of variant-supporting bases (reads2)',
       'Depth of reference-supporting bases on forward strand (reads1plus)',
       'Depth of reference-supporting bases on reverse strand (reads1minus)',
       'Depth of variant-supporting bases on forward strand (reads2plus)',
       'Depth of variant-supporting bases on reverse strand (reads2minus)',
       'Variant allele frequency', 'P-value from Fishers Exact Test', 'HET',
       'HOM', 'Zygosity', 'CSQ', 'csq', 'Consequence', 'Impact', 'Gene_Name',
       'Biotype', 'Strand', 'Clinical Significance', 'Codons_position',
       'Protein', 'Codon', 'Clinvar ID', 'Exon', 'Existing variant',
       'CLNREVSTAT', 'CLNDN', 'Position_14', 'Position_15',
       'Prot

In [26]:
df = vcf[['Gene_Name', 'ID', 'Consequence', 'Zygosity', 
          'CLNDN', 'Clinical Significance', 'CLNREVSTAT', 
          'Clinvar ID', 'Biotype', 'Strand', 'Protein Position and Amino Acid', 
          'Protein', 'Codon', 'Codons_position', 'CHROM', 'Exon', 'POS', 
          'REF', 'ALT', 'Impact', 'Existing variant', 'Genotype', 'Genotype Quality',
       'Raw Read Depth as reported by SAMtools',
       'Quality Read Depth of bases with Phred score >= 15',
       'Depth of reference-supporting bases (reads1)',
       'Depth of variant-supporting bases (reads2)',
       'Depth of reference-supporting bases on forward strand (reads1plus)',
       'Depth of reference-supporting bases on reverse strand (reads1minus)',
       'Depth of variant-supporting bases on forward strand (reads2plus)',
       'Depth of variant-supporting bases on reverse strand (reads2minus)',
       'Variant allele frequency', 'P-value from Fishers Exact Test']]
df

,Gene_Name,ID,Consequence,Zygosity,CLNDN,Clinical Significance,CLNREVSTAT,Clinvar ID,Biotype,Strand,...,Raw Read Depth as reported by SAMtools,Quality Read Depth of bases with Phred score >= 15,Depth of reference-supporting bases (reads1),Depth of variant-supporting bases (reads2),Depth of reference-supporting bases on forward strand (reads1plus),Depth of reference-supporting bases on reverse strand (reads1minus),Depth of variant-supporting bases on forward strand (reads2plus),Depth of variant-supporting bases on reverse strand (reads2minus),Variant allele frequency,P-value from Fishers Exact Test
0,DDX11L1,rs17855866,downstream_gene_variant,Heterozygous,,,,,transcribed_unprocessed_pseudogene,1,...,35,35,28,7,20%,0.005609,28,0,7,0
0,DDX11L2,rs17855866,downstream_gene_variant,Heterozygous,,,,,lncRNA,1,...,35,35,28,7,20%,0.005609,28,0,7,0
0,WASH7P,rs17855866,intron_variant&non_coding_transcript_variant,Heterozygous,,,,,unprocessed_pseudogene,-1,...,35,35,28,7,20%,0.005609,28,0,7,0
0,MIR6859-1,rs17855866,downstream_gene_variant,Heterozygous,,,,,miRNA,-1,...,35,35,28,7,20%,0.005609,28,0,7,0
0,,rs17855866,regulatory_region_variant,Heterozygous,,,,,enhancer,,...,35,35,28,7,20%,0.005609,28,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,WASH6P,.,non_coding_transcript_exon_variant,Homozygous,,,,,retained_intron,1,...,8,8,1,7,87.5%,0.000699,0,1,6,1
47668,WASH6P,.,non_coding_transcript_exon_variant,Homozygous,,,,,retained_intron,1,...,8,8,1,7,87.5%,0.000699,0,1,6,1
47668,WASH6P,.,downstream_gene_variant,Homozygous,,,,,retained_intron,1,...,8,8,1,7,87.5%,0.000699,0,1,6,1
47668,WASH6P,.,intron_variant&non_coding_transcript_variant,Homozygous,,,,,retained_intron,1,...,8,8,1,7,87.5%,0.000699,0,1,6,1


In [ ]:
df.to_excel(r'C:/Users/Dell/Downloads/KHHSPTGPTTL34_final_vcf.xlsx', index=False)

In [27]:
x = vcf[vcf['ID'] == 'rs6672356']
x

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,Protein,Codon,Clinvar ID,Exon,Existing variant,CLNREVSTAT,CLNDN,Position_14,Position_15,Protein Position and Amino Acid
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,ENSP00000349216.4:p.Trp251Arg,Tgg/Cgg,1166513,8/12,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,251,W/R,W251R
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,ENSP00000342313.3:p.Trp343Arg,Tgg/Cgg,1166513,10/14,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,343,W/R,W343R
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,ENSP00000412228.1:p.Trp170Arg,Tgg/Cgg,1166513,4/7,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,170,W/R,W170R
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,1/2,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,2/2,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,3/4,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN
30,chr1,942451,rs6672356,T,C,.,PASS,"ADP=26;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0,1;COMMO...",1/1,146,...,,,1166513,,rs6672356&COSV58985794,criteria_provided&_single_submitter,not_provided,,,NaN


In [33]:
vcf.columns

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO',
       'Genotype', 'Genotype Quality',
       'Raw Read Depth as reported by SAMtools',
       'Quality Read Depth of bases with Phred score >= 15',
       'Depth of reference-supporting bases (reads1)',
       'Depth of variant-supporting bases (reads2)',
       'Depth of reference-supporting bases on forward strand (reads1plus)',
       'Depth of reference-supporting bases on reverse strand (reads1minus)',
       'Depth of variant-supporting bases on forward strand (reads2plus)',
       'Depth of variant-supporting bases on reverse strand (reads2minus)',
       'Variant allele frequency', 'P-value from Fishers Exact Test', 'HET',
       'HOM', 'Zygosity', 'CSQ', 'csq', 'Consequence', 'Impact', 'Gene_Name',
       'Biotype', 'Strand', 'Clinical Significance', 'Codons_position',
       'Protein', 'Codon', 'Clinvar ID', 'Exon', 'Existing variant'],
      dtype='object')

In [18]:
df_adp = vcf.copy()
df_adp

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,


In [19]:
df_adp["New_Column"] = df_adp["INFO"].str.extract(r'^(.*?)(?=;?CSQ)')
df_adp

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Genotype,Genotype Quality,...,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN,New_Column
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
0,chr1,14752,rs17855866,G,A,.,PASS,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,22,...,,,,,,,,,,ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,ADP=8;WT=0;HET=0;HOM=1;NC=0
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,ADP=8;WT=0;HET=0;HOM=1;NC=0
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,ADP=8;WT=0;HET=0;HOM=1;NC=0
47668,chrY,57210016,.,C,T,.,PASS,ADP=8;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_varia...,1/1,31,...,,,,,,,,,,ADP=8;WT=0;HET=0;HOM=1;NC=0


In [21]:
df_adp.New_Column.iloc[0]

'ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX11L1:100287102|WASH7P:653635;R3;RS=17855866;RSPOS=14752;SAO=0;SSR=0;TOPMED=0.99442533129459734,0.00553484964322120,0.00003981906218144;VC=SNV;VP=0x050000040005000002000100;WGT=1;dbSNPBuildID=123'

In [22]:
vcf.csq.iloc[2]

'A|intron_variant&non_coding_transcript_variant|MODIFIER|WASH7P|ENSG00000227232|Transcript|ENST00000488147|unprocessed_pseudogene||10/10|ENST00000488147.1:n.1254-251C>T|||||||rs17855866||-1||SNV|HGNC|HGNC:38034|YES|||||||||||||||||||||||||||||||||0.00162|0.005487|0|0.001186|0.0002987|0.0001935|0|0|6.055e-05|0.00203|0.0002195|0.005487|gnomADg_AFR|||||||||||||'

In [17]:
vcf.csq.iloc[1]

'A|downstream_gene_variant|MODIFIER|DDX11L2|ENSG00000290825|Transcript|ENST00000456328|lncRNA||||||||||rs17855866|343|1||SNV|EntrezGene||YES|||1||||||||||||||||||||||||||||||0.00162|0.005487|0|0.001186|0.0002987|0.0001935|0|0|6.055e-05|0.00203|0.0002195|0.005487|gnomADg_AFR|||||||||||||'

In [18]:
vcf.csq.iloc[500]

'T|synonymous_variant|LOW|AGRN|ENSG00000188157|Transcript|ENST00000620552|protein_coding|39/39||ENST00000620552.4:c.5712C>T|ENSP00000484607.1:p.Asp1904%3D|6176|5712|1904|D|gaC/gaT|rs4275402&COSV65070964||1||SNV|HGNC|HGNC:329||||5|||ENSP00000484607||A0A087X208.61|UPI0004E4CB7F|||1|||Gene3D:2.60.120.200&Pfam:PF00054&PROSITE_profiles:PS50025&SMART:SM00282&Superfamily:SSF49899&CDD:cd00110|||0.5853|0.2617|0.7262|0.8214|0.6779|0.5838|0.6764|0.3114|0.7722|0.6575|0.8138|0.641|0.698|0.6758|0.6133|0.5913|0.3227|0.6733|0.7196|0.6622|0.8119|0.6343|0.6392|0.6961|0.5995|0.6072|0.8214|EAS|benign|0&1|1&1|25741868&18414213||||||128319|Benign|criteria_provided&_multiple_submitters&_no_conflicts|Congenital_myasthenic_syndrome_8&not_specified'

In [19]:
vcf.csq.iloc[2234]

'T|downstream_gene_variant|MODIFIER|RER1|ENSG00000157916|Transcript|ENST00000378518|protein_coding||||||||||rs1129171&COSV56579903|3268|1||SNV|HGNC|HGNC:30309||||3|||ENSP00000367779||Q5T093.117|UPI0000470C2F|||||||||0.3065|0.1815|0.268|0.4544|0.3062|0.3507||||||||||0.2822|0.1869|0.3169|0.2834|0.4237|0.4289|0.2518|0.269|0.3216|0.2989|0.334|0.4544|EAS|benign|0&1|1&1|||||||296268|Benign|criteria_provided&_multiple_submitters&_no_conflicts|not_provided&Peroxisome_biogenesis_disorder_6A_(Zellweger)'

In [16]:
df_1 = pd.read_csv(r'C:/Users/Dell/Downloads/test_file.csv', sep = '\t')
df_1

,CHROM,POS,rsID,REF,ALT,INFO,Final_Gene
0,chr1,15461640,rs372947070,C,T,RS=372947070;dbSNPBuildID=138;SSR=0;GENEINFO=C...,CELA2B
1,chr1,15461640,rs372947070,C,T,RS=372947070;dbSNPBuildID=138;SSR=0;GENEINFO=C...,CELA2A
2,chr1,65572326,rs35573508,A,"G,T",RS=35573508;dbSNPBuildID=126;SSR=0;GENEINFO=LE...,LEPR
3,chr1,65616108,rs34499590,C,"G,T",RS=34499590;dbSNPBuildID=126;SSR=0;GENEINFO=LE...,LEPR
4,chr1,65636215,rs142647085,A,"G,T",RS=142647085;dbSNPBuildID=134;SSR=0;GENEINFO=L...,LEPR
5,chr1,65636536,rs145651189,A,T,RS=145651189;dbSNPBuildID=134;SSR=0;GENEINFO=L...,LEPR
6,chr1,65636996,rs145685060,T,C,RS=145685060;dbSNPBuildID=134;SSR=0;GENEINFO=L...,LEPR
7,chr1,112918054,rs149491709,CAATAAATAAATAAATAAATAAATA,"C,CAATA,CAATAAATA,CAATAAATAAATA,CAATAAATAAATAA...",RS=149491709;dbSNPBuildID=134;SSR=0;GENEINFO=S...,SLC16A1
8,chr1,112918057,rs773587590,T,TAAAG,RS=773587590;dbSNPBuildID=144;SSR=0;GENEINFO=S...,SLC16A1
9,chr1,112956232,rs58047463,CG,"C,CGG",RS=58047463;dbSNPBuildID=129;SSR=0;GENEINFO=SL...,SLC16A1


In [17]:
df_1.INFO.iloc[0]

'RS=372947070;dbSNPBuildID=138;SSR=0;GENEINFO=CELA2A:63036;VC=SNV;NSM;GNO;FREQ=1000Genomes:0.9997,0.0003123|ExAC:0.9998,0.0002224|GnomAD:0.9999,7.128e-05|GnomAD_exomes:0.9998,0.0002108|GoESP:0.9999,7.689e-05|TOMMO:1,3.539e-05|TOPMED:0.9999,6.423e-05|dbGaP_PopFreq:0.9999,5.645e-05;CLNVI=.,;CLNORIGIN=.,1;CLNSIG=.,5|5;CLNDISDB=.,Office_of_Rare_Diseases:11944/MONDO:MONDO:0005010/MeSH:D003324/MedGen:C1956346/MONDO:MONDO:0005347/MedGen:C0813230/Human_Phenotype_Ontology:HP:0002155/Human_Phenotype_Ontology:HP:0003082/Human_Phenotype_Ontology:HP:0008174/Human_Phenotype_Ontology:HP:0008332/MONDO:MONDO:0005044/MedGen:C0020538/Human_Phenotype_Ontology:HP:0000822/Human_Phenotype_Ontology:HP:0004949/Human_Phenotype_Ontology:HP:0005126/MedGen:C1320657|MONDO:MONDO:0032837/MedGen:C5231430/OMIM:618620;CLNDN=.,Coronary_artery_disorder/Diabetes/Hypertensive_disorder/Hypertriglyceridemia|Abdominal_obesity-metabolic_syndrome_4;CLNREVSTAT=.,no_criteria|single;CLNACC=.,RCV000853074.1|RCV002249481.1;CLNHGVS=NC